In [67]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
from tqdm import tqdm

In [82]:
def calc_prob(channels, ydim, xdim, freqID, src_ID, noise, noise_level):
    
    ref = 'BON'
    
    #Pre-allocate array with probabilities
    probs_tw = np.zeros((xdim*ydim))
    
    # Energy ratios are calculated for each component in respect to the chosen
    # reference station
    for ch in channels:
        
        # Energy ratios from simulated signal
        if ch == 'Z':
            direc_comp = 'data/simu/vertZ/'
        elif ch == 'E':
            direc_comp = 'data/simu/horzE/'
        elif ch == 'N':
            direc_comp = 'data/simu/horzN/'
            
            
        if ch == 'Z':
            stas_name = ['BOR', 'DSO', 'SNE']
        else:
            stas_name = ['BOR', 'SNE']

        nRatios = len(stas_name)
        ratio_obs = np.zeros(nRatios)   
        
        path_simu_ref = direc_comp + ref + '/' 
        energy_simu_ref = np.loadtxt(path_simu_ref + freqID + '/energy_Fz.txt')
        #observed energy
        energy_curr_ref  = energy_simu_ref[src_ID]
        if noise:
            energy_curr_ref += np.abs(np.random.normal(0,noise_level))
        #simulated energy catalogue
        ###energy_simu_ref = np.reshape(energy_simu_ref,(ydim,xdim))
        ratios_simu = np.zeros((ydim*xdim,nRatios)) 
        
        for idxSrc, sta in enumerate(stas_name):
            path_simu = direc_comp  + sta + '/' 
            energy_simu = np.loadtxt(path_simu + freqID + '/energy_Fz.txt')
            #observed energy
            energy_curr = energy_simu[src_ID] 
            if noise:
                energy_curr += np.abs(np.random.normal(0,noise_level))
            ratio_obs[idxSrc] = energy_curr/energy_curr_ref
            #simulated energy catalogue
            ###energy_simu = np.reshape(energy_simu,(ydim,xdim))
            ratios_simu[:,idxSrc] = energy_simu/energy_simu_ref
                    
        # Error between observed and simulated energy ratios
        for ii in range(xdim*ydim):
            for kk in range(nRatios):
                probs_tw[ii] += 1./nRatios * (
                    np.abs(np.log10(ratios_simu[ii,kk]/ratio_obs[kk])))

    # Probability is defined as inverse of error, normalized by 
    # number of channels
    probs_tw = 1./(probs_tw/len(channels))

    return(probs_tw)

In [83]:
def exponential(x, a, k, b):
    return a*np.exp(x*k) + b

In [89]:
noise = False

channels = ['Z']
freqID   = 'BP8to12'

# noise level
if freqID == 'BP3to7':
    noise_level = 1.025e-24
elif freqID == 'BP8to12':
    noise_level = 1.050e-24
elif freqID == 'BP13to17':
    noise_level = 3.277e-26
    
    
xdim = 121
ydim = 101

## Load topography
topo   = np.loadtxt('data/DEM_PF_10m_cut.dat',skiprows=6)
dx = 10
dy = 10
xx = np.arange(0.,2100.+dx,dx)
yy = np.arange(0.,1800.+dy,dy)
levels = np.arange(2190., 2650, 20)
### Load stations
srcs   = np.loadtxt('data/stations.txt')
srcs_array = ['BON', 'BOR', 'DSO', 'SNE']
xmin =  640.
xmax = 1840.
ymin =  400.
ymax = 1400.

# Loading grid positions
data = pd.read_csv('data/positions.txt', sep='\s+', 
                   header=None, names=['x', 'y', 'z']) 

In [94]:
data

,x,y,z,res_a,res_a_err,res_k,res_k_err,res_b,res_b_err
0,640.0,400.0,671.8667,28.222763,0.325559,-0.018798,0.000187,0.607923,0.006384
1,650.0,400.0,672.2667,23.039543,0.208644,-0.015724,0.000130,0.592287,0.005517
2,660.0,400.0,672.6667,21.563709,0.209436,-0.016321,0.000149,0.598978,0.005689
3,670.0,400.0,673.0667,44.730404,0.516055,-0.031618,0.000334,0.650724,0.007321
4,680.0,400.0,673.8667,59.796295,0.709342,-0.042727,0.000433,0.665683,0.007225
...,...,...,...,...,...,...,...,...,...
12216,1800.0,1400.0,611.2667,8.621846,0.125869,-0.006565,0.000099,0.372287,0.009632
12217,1810.0,1400.0,606.8667,9.918320,0.105117,-0.006717,0.000072,0.392855,0.007563
12218,1820.0,1400.0,603.2667,11.121420,0.131066,-0.007340,0.000084,0.425141,0.008115
12219,1830.0,1400.0,598.8667,6.937831,0.088056,-0.005860,0.000075,0.404518,0.006959


In [90]:
#resolution = np.zeros((xdim*ydim))
resolution_popt = np.zeros((xdim*ydim, 3))
resolution_perr = np.zeros((xdim*ydim, 3))

for index, row in tqdm(data.iterrows()):
    
    radius = np.array(np.sqrt((data.x-row.x)**2 + (data.y-row.y)**2))
    probs  = calc_prob(channels, ydim, xdim, freqID, index, noise, noise_level)
    
    radius = np.delete(radius, index)
    probs  = np.delete(probs,  index)
    
    try:
        popt_exponential, pcov_exponential = optimize.curve_fit(exponential, radius, probs, p0=[10, -0.5, 0])
        perr_exponential = np.sqrt(np.diag(pcov_exponential))
        
        resolution_popt[index,:] = popt_exponential
        resolution_perr[index,:] = perr_exponential
        
        #y0 = exponential(0., popt_exponential[0], popt_exponential[1], popt_exponential[2])
        #xhalf = np.log((y0/2 - popt_exponential[2])/popt_exponential[0])/popt_exponential[1]
    except RuntimeError:
        print('Fit not found:')
        print(index)
        resolution_popt[index,:] = [np.nan, np.nan, np.nan]
        resolution_perr[index,:] = [np.nan, np.nan, np.nan]

0it [00:00, ?it/s]/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in true_divide
/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in multiply
  
18it [00:05,  3.60it/s]/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
220it [01:10,  2.75it/s]

Fit not found:
219


1135it [06:03,  2.79it/s]

Fit not found:
1134


1249it [06:40,  2.85it/s]

...

7808it [40:32,  2.64it/s]

Fit not found:
7807


8542it [44:20,  1.96it/s]

Fit not found:
8541


12221it [1:04:41,  3.15it/s]


In [91]:
#data['res'] = resolution
data['res_a']     = resolution_popt[:,0]
data['res_a_err'] = resolution_perr[:,0]
data['res_k']     = resolution_popt[:,1]
data['res_k_err'] = resolution_perr[:,1]
data['res_b']     = resolution_popt[:,2]
data['res_b_err'] = resolution_perr[:,2]

In [92]:
data.head()

,x,y,z,res_a,res_a_err,res_k,res_k_err,res_b,res_b_err
0,640.0,400.0,671.8667,28.222763,0.325559,-0.018798,0.000187,0.607923,0.006384
1,650.0,400.0,672.2667,23.039543,0.208644,-0.015724,0.000130,0.592287,0.005517
2,660.0,400.0,672.6667,21.563709,0.209436,-0.016321,0.000149,0.598978,0.005689
3,670.0,400.0,673.0667,44.730404,0.516055,-0.031618,0.000334,0.650724,0.007321
4,680.0,400.0,673.8667,59.796295,0.709342,-0.042727,0.000433,0.665683,0.007225


In [93]:
data.to_csv('resolution/resolution_exp_Z_'+freqID+'.csv', index=False)